In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import cudf as cd
import cupy as cp

In [3]:
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve
import sys
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

from sklearn.metrics import mean_squared_error as MSE

In [4]:
from cuml.preprocessing.LabelEncoder import LabelEncoder
from cuml.model_selection import train_test_split
from cuml.metrics.regression import r2_score,mean_squared_log_error,mean_squared_error
from cuml.metrics import confusion_matrix,roc_auc_score,precision_recall_curve
from cuml.metrics.accuracy import accuracy_score
from cuml import ElasticNet
from cuml.ensemble import RandomForestRegressor
import xgboost as xgb

In [5]:
train = pd.read_csv("../input/ipl-2020-player-performance/Training.csv",header=0)
cd_train = cd.read_csv("../input/ipl-2020-player-performance/Training.csv",header=0)
cd_train.head()

In [6]:
cd_train.info()

In [7]:
train['Match_ID'] = train['Id']
train['Player'] = train['Id']
cd_train['Match_ID'] = cd_train['Id']
cd_train['Player'] = cd_train['Id']
for i in range(0, len( train)):
    train['Player'][i] =  train['Id'][i].split("_")[-1]
    train['Match_ID'][i] = int( train['Id'][i].split('_')[:1][0])
cd_train['Player']=cd.Series(train['Player'])
cd_train['Match_ID']=cd.Series(train['Match_ID'])


In [8]:
cd_train.head()

In [9]:
columns = ['Match_ID','Player','Total Points']
train = train[columns]
cd_train = cd_train[columns]
cd_train.head()

In [10]:
submission = pd.read_csv("../input/ipl-2020-player-performance/sample_submission.csv",header=0)
cd_submission = cd.read_csv("../input/ipl-2020-player-performance/sample_submission.csv",header=0)
cd_submission.head()

In [11]:
submission['Match_ID'] = submission['Id']
submission['Player'] = submission['Id']
cd_submission['Match_ID'] = cd_submission['Id']
cd_submission['Player'] = cd_submission['Id']
for i in range(0, len( submission)):
    submission['Player'][i] =  submission['Id'][i].split("_")[-1]
    submission['Match_ID'][i] = int( submission['Id'][i].split('_')[:1][0])
    
cd_submission['Player']=cd.Series(submission['Player'])
cd_submission['Match_ID']=cd.Series(submission['Match_ID'])

In [12]:
submission = submission[columns]
cd_submission = cd_submission[columns]
cd_submission.head()

In [13]:
match8_19 = pd.read_csv("../input/ipl-2020-player-performance/Matches IPL 2008-2019.csv",header=0)
match20 = pd.read_csv("../input/ipl-2020-player-performance/Matches IPL 2020.csv",header=0)
cd_match8_19 = cd.read_csv("../input/ipl-2020-player-performance/Matches IPL 2008-2019.csv",header=0)
cd_match20 = cd.read_csv("../input/ipl-2020-player-performance/Matches IPL 2020.csv",header=0)
match8_19.head()

In [14]:
cd_match8_19.info()

In [15]:
teams = match8_19.team1.unique()
teams2 = match20.team1.unique()
cd_teams = cd_match8_19.team1.unique()
cd_teams2 = cd_match20.team1.unique()

print(cd_teams)
print(cd_teams2)

In [16]:
short = { 'Kolkata Knight Riders':'KKR','Chennai Super Kings':'CSK','Rajasthan Royals':'RR',
         'Mumbai Indians':'MI','Deccan Chargers':'SRH','Kings XI Punjab':'KXIP',
         'Royal Challengers Bangalore':'RCB','Delhi Daredevils':'DC',
         'Sunrisers Hyderabad':'SRH','Delhi Capitals':'DC','Rising Pune Supergiants':'RPS',
         'Gujarat Lions':'GL','Rising Pune Supergiant':'RPS','Pune Warriors':'RPS',
         'Kochi Tuskers Kerala':'KTK'}

In [17]:
match8_19['team1'] = match8_19['team1'].map(short) 
match8_19['team2'] = match8_19['team2'].map(short)
cd_match8_19['team1'] = cd_match8_19['team1'].map(short) 
cd_match8_19['team2'] = cd_match8_19['team2'].map(short)
cd_match8_19.head()

In [18]:
match8_19.rename(columns = {'match_id':'Match_ID'}, inplace = True) 
cd_match8_19.rename(columns = {'match_id':'Match_ID'}, inplace = True) 
cd_match8_19.head()

In [19]:
train_m = pd.merge(train[['Match_ID','Player','Total Points']], match8_19[['Match_ID','team1','team2']], on = 'Match_ID', how='inner')
cd_train_m = cd.merge(cd_train[['Match_ID','Player','Total Points']], cd_match8_19[['Match_ID','team1','team2']], on = 'Match_ID', how='inner')
cd_train_m.head()

In [20]:
match20.rename(columns = {'match_id':'Match_ID'}, inplace = True)
cd_match20.rename(columns = {'match_id':'Match_ID'}, inplace = True) 
cd_match20.head()

In [21]:
submit_m = pd.merge(submission[['Match_ID','Player','Total Points']], match20[['Match_ID','team1','team2']], on = 'Match_ID', how='inner')
cd_submit_m = cd.merge(cd_submission[['Match_ID','Player','Total Points']], cd_match20[['Match_ID','team1','team2']], on = 'Match_ID', how='inner')
cd_submit_m.head()

In [22]:
all_added = pd.concat([train_m,submit_m]) 
cd_all_added = cd.concat([cd_train_m,cd_submit_m]) 
cd_all_added.info()

In [23]:
Avg_scores = all_added.groupby('Player')['Total Points'].mean().reset_index()
Avg_scores.rename(columns = {'Total Points':'Avg'}, inplace = True) 
cd_Avg_scores = cd_all_added.groupby('Player')['Total Points'].mean().reset_index()
cd_Avg_scores.rename(columns = {'Total Points':'Avg'}, inplace = True) 
cd_Avg_scores.head()

In [24]:
all_added1 = pd.merge(all_added[['Player','Match_ID','team1','team2']], Avg_scores[['Player','Avg']], on = 'Player', how='inner')
cd_all_added1 = cd.merge(cd_all_added[['Player','Match_ID','team1','team2']], cd_Avg_scores[['Player','Avg']], on = 'Player', how='inner')
print(cd_all_added1.shape)
cd_all_added1.head()

In [25]:
final_final = pd.merge(all_added[['Match_ID','Player']],all_added1[['Player','Match_ID','team1','team2','Avg']], on = ['Match_ID','Player'], how='inner')
cd_final_final = cd.merge(cd_all_added[['Match_ID','Player']],cd_all_added1[['Player','Match_ID','team1','team2','Avg']], on = ['Match_ID','Player'], how='inner')
print(cd_final_final.shape)
cd_final_final.head()

In [26]:
X = final_final[['Player','team1','team2','Avg']]
y = train['Total Points']
cd_X = cd_final_final[['Player','team1','team2','Avg']]
cd_y = cd_train['Total Points']
cd_X.head()

In [27]:
label=X.select_dtypes(include='object')
cd_label=cd_X.select_dtypes(include='object')
cd_label

In [28]:
labelencoder = LabelEncoder()
cd_label['Player']=labelencoder.fit_transform(cd_label['Player'])
cd_label['team1']=labelencoder.fit_transform(cd_label['team1'])
cd_label['team2']=labelencoder.fit_transform(cd_label['team2'])
cd_label['Avg']=cd_X['Avg']

In [29]:
train_enc = cd_label[:15916]
sub_enc = cd_label[15916:]

In [30]:
train_enc.shape,sub_enc.shape,cd_y.shape

In [31]:
for col in train_enc.columns:
    a = train_enc[col].mean()
    b = train_enc[col].std()
    train_enc[col]=train_enc[col].applymap(lambda x: ((x-a)/b))
train_enc = train_enc.astype('float32')

for col in sub_enc.columns:
    a = sub_enc[col].mean()
    b = sub_enc[col].std()
    sub_enc[col] = sub_enc[col].applymap(lambda x: ((x-a)/b))
sub_enc = sub_enc.astype('float32')

In [32]:
import scipy.stats as stats
import pylab

In [33]:
minimum = cd_y.min().copy()
yy = (cd_y.values.get()-minimum)**(1/2)
print(pd.Series(yy).skew())
sns.kdeplot(yy)

In [34]:
sns.countplot(yy)

In [35]:
np.unique(yy==np.nan,return_counts=True)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(train_enc, cp.array(yy), test_size=0.2, random_state=25)

In [37]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [38]:
X_train

In [39]:
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [40]:
reg = ElasticNet(alpha=0.5)
reg.fit(X_train,y_train)

reg.score(X_test,y_test),r2_score(y_test,reg.predict(X_test)),accuracy_score(y_test,(reg.predict(X_test))),accuracy_score(y_test,(reg.predict(X_test))**3),mean_squared_error(y_test,(reg.predict(X_test))),(mean_squared_error(y_test,reg.predict(X_test)))**1/2,(mean_squared_log_error(y_test,reg.predict(X_test)))

In [41]:
reg = RandomForestRegressor(n_estimators=400,max_depth=16)
reg.fit(X_train,y_train)
reg.score(X_test,y_test),r2_score(y_test,reg.predict(X_test)),accuracy_score(y_test,(reg.predict(X_test))),mean_squared_error(y_test,(reg.predict(X_test)))**1/2,(mean_squared_log_error(y_test,reg.predict(X_test)))

In [43]:
y_pred=cp.round(reg.predict(X_test))

In [44]:
xgb_reg = xgb.XGBRegressor(max_depth=3, n_estimators=250, learning_rate=0.01,
                            n_jobs=-1,random_state=2519,subsample=0.8)

In [45]:
xgb_reg.fit(X_train, y_train)

In [46]:
r2_score(y_test,xgb_reg.predict(X_test)),accuracy_score(y_test,xgb_reg.predict(X_test)),(mean_squared_error(y_test,xgb_reg.predict(X_test)))**1/2,(mean_squared_log_error(y_test,xgb_reg.predict(X_test)))

In [47]:


params = {
   
    'max_depth': 3,
    'subsample': 0.8,
    'colsample_bytree': 0.9, 
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    'learning_rate': 0.01
}



In [48]:
def XGBmodel(X_train,X_test,y_train,y_test,params,val):
    matrix_train = xgb.DMatrix(X_train,label=y_train)
    matrix_test = xgb.DMatrix(X_test,label=y_test)
    if val:
        model=xgb.train(params=params,
                        dtrain=matrix_train,num_boost_round=5000, 
                        early_stopping_rounds=10,evals=[(matrix_test,'test')])
    else:
        model=xgb.train(params=params,dtrain=matrix_train,num_boost_round=5000)
    return model

model = XGBmodel(X_train,X_test,y_train,y_test,params,True)

In [49]:
prediction = model.predict(xgb.DMatrix(X_test), ntree_limit = model.best_ntree_limit).tolist()

In [50]:
prediction = cp.round(prediction)
prediction

In [51]:
model = XGBmodel(train_enc,X_test,cp.array(yy),y_test,params,False)
prediction = model.predict(xgb.DMatrix(sub_enc), ntree_limit = model.best_ntree_limit).tolist()
prediction = cp.round(cp.array(prediction)**2+minimum)
prediction

In [53]:
prediction = [10 if i < 0 else i for i in prediction]

In [54]:
sub = cd.read_csv("../input/ipl-2020-player-performance/sample_submission.csv",header=0)
submission1 = sub.copy()
submission1.head()

In [55]:
y_pred = [10 if i<0 else i for i in y_pred]

In [56]:
submission1['Total Points']=prediction

In [57]:
submission1.to_pandas().to_csv("submission.csv", index=False)